### Libraries:

In [1]:
# Core libraries:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from io import BytesIO
from zipfile import ZipFile
import requests
import warnings
warnings.filterwarnings("ignore")

# ML libraries:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import MinMaxScaler

### Import data:

In [2]:
url = 'https://github.com/manugaco/ML_User_Cases/blob/master/Datasets/RFM_data.zip?raw=true'
zip = ZipFile(BytesIO(requests.get(url).content), 'r')
df = pd.read_csv(zip.open(zip.namelist()[0]), low_memory=False, encoding="ISO-8859-1", dtype={'CustomerID': str,'InvoiceID': str})

### Data Exploration:

In [3]:
df.shape

(541909, 8)

In [4]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850,United Kingdom


#### Missing values:

In [5]:
df.isna().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

Variables Description and CustomerID have missing values. The first one is not important but the second one is, so I drop all of these NaN rows.

In [6]:
df = df.dropna(subset=['CustomerID'])

#### Duplicated entries:

In [7]:
df.duplicated().sum()

5225

There are duplicated entries in the dataset which might not add any relevant information. I remove those.

In [8]:
df = df.drop_duplicates()

#### Data Quality:

##### Features ranges:

In [9]:
df.StockCode.value_counts()

85123A    2065
22423     1894
85099B    1659
47566     1409
84879     1405
          ... 
84963A       1
90100        1
84569C       1
90152B       1
90102        1
Name: StockCode, Length: 3684, dtype: int64

In [10]:
df.Country.value_counts()

United Kingdom          356728
Germany                   9480
France                    8475
EIRE                      7475
Spain                     2528
Netherlands               2371
Belgium                   2069
Switzerland               1877
Portugal                  1471
Australia                 1258
Norway                    1086
Italy                      803
Channel Islands            757
Finland                    695
Cyprus                     611
Sweden                     461
Austria                    401
Denmark                    389
Japan                      358
Poland                     341
USA                        291
Israel                     247
Unspecified                241
Singapore                  229
Iceland                    182
Canada                     151
Greece                     146
Malta                      127
United Arab Emirates        68
European Community          61
RSA                         58
Lebanon                     45
Lithuani

In [11]:
df.Quantity.value_counts()

 1      69605
 12     59828
 2      57425
 6      37480
 4      32093
        ...  
 701        1
-66         1
 101        1
-670        1
 348        1
Name: Quantity, Length: 436, dtype: int64

Quantity has negative values which makes no sense. I remove all values =< 0.

In [12]:
df = df[df['Quantity']>=0]

In [13]:
df.Quantity.value_counts()

1       69605
12      59828
2       57425
6       37480
4       32093
        ...  
824         1
1878        1
157         1
3186        1
87          1
Name: Quantity, Length: 302, dtype: int64

In [14]:
df.InvoiceDate.astype('datetime64[ns]').min()

Timestamp('2010-12-01 08:26:00')

In [15]:
df.InvoiceDate.astype('datetime64[ns]').max()

Timestamp('2011-12-09 12:50:00')

### RFM Segmentation:

#### Recency

##### Number of days since last purchase:

In [16]:
df['InvoiceDate'] = df['InvoiceDate'].astype('datetime64[ns]')
last_day = df['InvoiceDate'].max()
df_rec = df[['CustomerID', 'InvoiceDate']]
df_rec = df_rec.groupby('CustomerID')['InvoiceDate'].max().reset_index()
df_rec['last_date'] = last_day
df_rec['Recency'] = df_rec['last_date'] - df_rec['InvoiceDate']
df_rec['Recency'] = df_rec['Recency'].dt.days
df_rec = df_rec[['CustomerID', 'Recency']]


In [17]:
df_rec.head()

,CustomerID,Recency
0,12346,325
1,12347,1
2,12348,74
3,12349,18
4,12350,309


#### Frequency

##### Number of times a customer purchases:

In [18]:
df_freq = df[['CustomerID', 'InvoiceNo']]
df_freq = df_freq.drop_duplicates()
df_freq = df_freq.groupby('CustomerID')['InvoiceNo'].count().reset_index()
df_freq = df_freq.rename(columns={'InvoiceNo':'Frequency'})

In [19]:
df_freq.head()

,CustomerID,Frequency
0,12346,1
1,12347,7
2,12348,4
3,12349,1
4,12350,1


#### Monetary

##### Total amount a customer spent:

In [20]:
df_mon = df[['CustomerID', 'Quantity', 'UnitPrice']]
df_mon['Monetary'] = df_mon['Quantity']*df_mon['UnitPrice']
df_mon = df_mon.groupby('CustomerID')['Monetary'].sum().reset_index()

In [21]:
df_mon.head()

,CustomerID,Monetary
0,12346,77183.60
1,12347,4310.00
2,12348,1797.24
3,12349,1757.55
4,12350,334.40


#### Modeling RFM table:

In [22]:
df_RFM = df_rec.merge(df_freq, on = 'CustomerID').merge(df_mon, on = 'CustomerID')
df_RFM = df_RFM.set_index('CustomerID')

In [23]:
df_RFM.head()

,Recency,Frequency,Monetary
CustomerID,,,
12346,325,1,77183.60
12347,1,7,4310.00
12348,74,4,1797.24
12349,18,1,1757.55
12350,309,1,334.40


In [24]:
df_RFM.shape

(4339, 3)

In [25]:
df_RFM.isna().sum()

Recency      0
Frequency    0
Monetary     0
dtype: int64

In [26]:
df_RFM.describe()

,Recency,Frequency,Monetary
count,4339.000000,4339.000000,4339.000000
mean,91.518322,4.271952,2048.215924
std,100.009747,7.705493,8984.248352
min,0.000000,1.000000,0.000000
25%,17.000000,1.000000,306.455000
50%,50.000000,2.000000,668.560000
75%,141.000000,5.000000,1660.315000
max,373.000000,210.000000,280206.020000


In [27]:
# Get rid off the outliers (to be analyzed in other post)

df_RFM = df_RFM[df_RFM['Recency']<df_RFM['Recency'].quantile(0.99)]
df_RFM = df_RFM[df_RFM['Frequency']<df_RFM['Frequency'].quantile(0.99)]
df_RFM = df_RFM[df_RFM['Monetary']<df_RFM['Monetary'].quantile(0.99)]
df_RFM

,Recency,Frequency,Monetary
CustomerID,,,
12347,1,7,4310.00
12348,74,4,1797.24
12349,18,1,1757.55
12350,309,1,334.40
12352,35,8,2506.04
...,...,...,...
18280,277,1,180.60
18281,180,1,80.82
18282,7,2,178.05


#### Kmeans segmentation:

##### Finding the optimum number of clusters by using the silhouette coefficient:

In [28]:
# Data is scaled by using the min max function.
df_RFM_scaled = MinMaxScaler().fit_transform(df_RFM)
df_RFM_scaled = pd.DataFrame(df_RFM_scaled).rename(columns={0:'Recency_sc', 1:'Frequency_sc', 2:'Monetary_sc'})

In [29]:
# Function to get the optimum number of clusters

def get_opt_k(data, column):

    range_c = range(2, 10)
    clusters = data[[column]]
    s_avg = []
    for k in range_c:
        kmeans = KMeans(n_clusters=k, max_iter=100).fit(clusters)
        s_avg.append(silhouette_score(clusters, kmeans.labels_ ))
    result = {range_c[i]: s_avg[i] for i in range(len(range_c))}
    k = max(result, key=result.get)
    print('Number of optimum '+ column + ' clusters: ', k)
    return k

In [30]:
rec_opt_k = get_opt_k(df_RFM_scaled, 'Recency_sc')
frq_opt_k = get_opt_k(df_RFM_scaled, 'Frequency_sc')
mon_opt_k = get_opt_k(df_RFM_scaled, 'Monetary_sc')

In [40]:
printrec_opt_k
frq_opt_k
mon_opt_k

2

##### Computing the clustering tags:

In [57]:
# Function to compute the kmeans and add the tag to the dataframe:

def comp_labels(data, column, opt_k):

    kmeans = KMeans(n_clusters=opt_k)
    kmeans.fit(data[[column]])
    data[column+'_clus_labels'] = kmeans.predict(data[[column]])

In [58]:
rfm_names = ['Recency', 'Frequency', 'Monetary']

for name in rfm_names:

    comp_labels(df_RFM, name, 3) # Number of cluster set to 3 insted of the optimum, to reduce the number of groups in the labeling process and type of customer.
    zero = df_RFM[df_RFM[name+'_clus_labels'] == 0][name]
    one = df_RFM[df_RFM[name+'_clus_labels'] == 1][name]
    two = df_RFM[df_RFM[name+'_clus_labels'] == 2][name]
    df_RFM[name+'_tag'] = np.where(df_RFM[name+'_clus_labels']==0, str(round(zero.min()))+'_to_'+str(round(zero.max())),
                            np.where(df_RFM[name+'_clus_labels']==1, str(round(one.min()))+'_to_'+str(round(one.max())),
                            np.where(df_RFM[name+'_clus_labels']==2, str(round(two.min()))+'_to_'+str(round(two.max())), 'none'
    )))

df_RFM.head()

,Recency,Frequency,Monetary,Recency_clus_labels,Frequency_clus_labels,Monetary_clus_labels,Recency_tag,Frequency_tag,Monetary_tag
CustomerID,,,,,,,,,
12347,1,7,4310.00,1,2,2,0_to_89,5_to_10,1756_to_5232
12348,74,4,1797.24,1,0,2,0_to_89,1_to_4,1756_to_5232
12349,18,1,1757.55,1,0,2,0_to_89,1_to_4,1756_to_5232
12350,309,1,334.40,0,0,0,215_to_367,1_to_4,0_to_1751
12352,35,8,2506.04,1,2,2,0_to_89,5_to_10,1756_to_5232


##### Distribution of the clusters amongst the RFM categories:

In [63]:
## Contingency table:

pd.crosstab([df_RFM['Recency_tag'], df_RFM['Frequency_tag']], df_RFM['Monetary_tag'])

Monetary_tag               0_to_1751  1756_to_5232  5291_to_12602
Recency_tag Frequency_tag                                        
0_to_89     11_to_28               3           142            106
            1_to_4              1676           156             14
            5_to_10              266           403             39
215_to_367  11_to_28               0             1              0
            1_to_4               596            12              0
            5_to_10                6             1              1
90_to_214   11_to_28               0             4              1
            1_to_4               685            33              1
            5_to_10               32            22              3

##### Computing the RFM score and assigning clients rating:

Key rules to score the customer value. These range come from the bounds of each cluster.

The lower the recency, the higher the value of the customer:
- Recency ranges: 0-89 = 2; 90-214 = 1; 215-367 = 0.

The higher the frequency, the higher the value of the customer:
- Frequency ranges: 1-4 = 0; 5-10 = 1; 11-28 = 2.

The higher the monetary, the higher the value of the customer:
- Monetary ranges: 0-1751 = 0; 1756-5232 = 1; 5291-12602 = 2

In [120]:
df_RFM['Recency_temp'] = np.where(df_RFM['Recency_clus_labels'] == 0, 0,
np.where(df_RFM['Recency_clus_labels'] == 1, 2,
np.where(df_RFM['Recency_clus_labels'] == 2, 1, 'none'
)))

df_RFM['Frequency_temp'] = np.where(df_RFM['Frequency_clus_labels'] == 0, 0,
np.where(df_RFM['Frequency_clus_labels'] == 1, 2,
np.where(df_RFM['Frequency_clus_labels'] == 2, 1, 'none'
)))

df_RFM['Monetary_temp'] = np.where(df_RFM['Monetary_clus_labels'] == 0, 0,
np.where(df_RFM['Monetary_clus_labels'] == 1, 2,
np.where(df_RFM['Monetary_clus_labels'] == 2, 1, 'none'
)))

df_RFM['RFM_score'] = df_RFM[['Recency_temp', 'Frequency_temp', 'Monetary_temp']].astype(str).apply(''.join, axis=1)

In [121]:
df_RFM.head()

,Recency,Frequency,Monetary,Recency_clus_labels,Frequency_clus_labels,Monetary_clus_labels,Recency_tag,Frequency_tag,Monetary_tag,RFM_score,Customer_labels,Recency_temp,Frequency_temp,Monetary_temp
CustomerID,,,,,,,,,,,,,,
12347,1,7,4310.00,1,2,2,0_to_89,5_to_10,1756_to_5232,211,Rest_customers,2,1,1
12348,74,4,1797.24,1,0,2,0_to_89,1_to_4,1756_to_5232,201,New_customer,2,0,1
12349,18,1,1757.55,1,0,2,0_to_89,1_to_4,1756_to_5232,201,New_customer,2,0,1
12350,309,1,334.40,0,0,0,215_to_367,1_to_4,0_to_1751,000,Churn_customer,0,0,0
12352,35,8,2506.04,1,2,2,0_to_89,5_to_10,1756_to_5232,211,Rest_customers,2,1,1


##### Grouping the number of customer profiles to 6:

1. Best_customers = 222
2. Frequent_customers = X2X
3. Cow_customers = XX2
4. New_customers = 20X
5. Churn_customers = 00X
6. Rest


In [122]:
df_RFM['Customer_labels'] = np.where((df_RFM['Recency_temp'] + df_RFM['Frequency_temp'] + df_RFM['Monetary_temp']) == '222', 'Best_customers',
np.where(df_RFM['Frequency_temp']=='2', 'Frequent_customers',
np.where(df_RFM['Monetary_temp']=='2', 'Cow_customers',
np.where(((df_RFM['Recency_temp']=='2') & (df_RFM['Frequency_temp']=='0')), 'New_customers',
np.where(((df_RFM['Recency_temp']=='0') & (df_RFM['Frequency_temp']=='0')), 'Churn_customers', 'Rest_customers'
)))))

df_RFM.drop(['Recency_temp', 'Frequency_temp', 'Monetary_temp'], axis=1, inplace=True)

##### Creating KPI's on the customers table:

In [123]:
df_RFM['Customer_labels'].value_counts()

New_customers         1832
Rest_customers        1448
Churn_customers        608
Frequent_customers     151
Best_customers         106
Cow_customers           58
Name: Customer_labels, dtype: int64

In [149]:
df_count = pd.DataFrame(df_RFM.groupby('Customer_labels').size()).reset_index().rename(columns={0:'Customer_count'})
df_rec_m = pd.DataFrame(df_RFM.groupby('Customer_labels')['Recency'].mean()).reset_index().rename(columns={'Recency':'Recency_mean'})
df_frq_m = pd.DataFrame(df_RFM.groupby('Customer_labels')['Frequency'].mean()).reset_index().rename(columns={'Frequency':'Frequency_mean'})
df_atv_m = pd.DataFrame(df_RFM.groupby('Customer_labels')['Monetary'].mean()).reset_index().rename(columns={'Monetary':'ATV_mean'})
df_sales = pd.DataFrame(round(df_RFM.groupby('Customer_labels')['Monetary'].sum()/df_RFM['Monetary'].sum()*100, 2)).reset_index().rename(columns={'Monetary':'%_sales'})
df_cust = pd.DataFrame(round(df_RFM.groupby('Customer_labels').size()/df_RFM.shape[0]*100, 2)).reset_index().rename(columns={0:'%_customers'})
df_resume = df_count.merge(df_rec_m).merge(df_frq_m).merge(df_atv_m).merge(df_sales).merge(df_cust)

In [150]:
df_resume

,Customer_labels,Customer_count,Recency_mean,Frequency_mean,ATV_mean,%_sales,%_customers
0,Best_customers,106,12.490566,17.141509,7756.684811,15.10,2.52
1,Churn_customers,608,282.478618,1.282895,398.672452,4.45,14.47
2,Cow_customers,58,39.827586,6.655172,7375.027069,7.85,1.38
3,Frequent_customers,151,19.158940,13.794702,3615.085894,10.02,3.59
4,New_customers,1832,37.102620,2.182314,766.817496,25.79,43.59
5,Rest_customers,1448,90.957182,4.256215,1383.569834,36.78,34.45
